In [1]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split



In [2]:

# 1. Charger et préparer les données
data = load_breast_cancer()
X, y = data.data, data.target
X = StandardScaler().fit_transform(X)


In [3]:

# Séparer pour évaluation finale
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 🟢 2. Entraînement du modèle centralisé (normal)
central_model = LogisticRegression(solver='lbfgs', max_iter=1000)
central_model.fit(X_train_full, y_train_full)


LogisticRegression(max_iter=1000)

In [4]:

central_coef = central_model.coef_.flatten()
central_intercept = central_model.intercept_
y_pred_central = central_model.predict(X_test)
central_acc = accuracy_score(y_test, y_pred_central)


In [5]:

# 🔒 3. Simuler 3 clients (ex : 3 hôpitaux)
num_clients = 3
client_data = np.array_split(X_train_full, num_clients)
client_labels = np.array_split(y_train_full, num_clients)

# 🔒 4. Entraînement local + agrégation sécurisée
def train_local_model(X_local, y_local):
    model = LogisticRegression(solver='lbfgs', max_iter=1000)
    model.fit(X_local, y_local)
    return model.coef_.flatten(), model.intercept_

coefs, intercepts = [], []
for i in range(num_clients):
    coef, intercept = train_local_model(client_data[i], client_labels[i])
    coefs.append(coef)
    intercepts.append(intercept)


In [6]:

# 🔒 5. Moyenne des poids (agrégation sécurisée)
global_coef = np.mean(coefs, axis=0)
global_intercept = np.mean(intercepts, axis=0)



In [7]:
# 🔒 6. Évaluer le modèle agrégé
federated_model = LogisticRegression()
federated_model.coef_ = global_coef.reshape(1, -1)
federated_model.intercept_ = global_intercept
federated_model.classes_ = np.array([0, 1])

y_pred_federated = federated_model.predict(X_test)
federated_acc = accuracy_score(y_test, y_pred_federated)


In [8]:

# 🧾 Résultats comparés
print("\n=========== ✅ MODÈLE CENTRALISÉ ===========")
print(f"Accuracy : {central_acc:.4f}")
print("Coefficients :")
print(central_coef)
print("Intercept :")
print(central_intercept)




=========== ✅ MODÈLE CENTRALISÉ ===========
Accuracy : 0.9737
Coefficients :
[-0.43464701 -0.39719369 -0.39606985 -0.46999027 -0.06739432  0.52671658
 -0.80767618 -1.1077114   0.24153785  0.07844689 -1.25057276  0.1889927
 -0.58988574 -0.91987672 -0.31656977  0.66906627  0.17107672 -0.31431731
  0.50499861  0.61176062 -0.87394431 -1.35751246 -0.58604724 -0.83803013
 -0.54737794  0.00589749 -0.95168648 -0.78040785 -1.19559714 -0.16293183]
Intercept :
[0.32857199]


In [9]:
print("\n=========== 🔒 MODÈLE FÉDÉRÉ (AGRÉGATION SÉCURISÉE) ===========")
print(f"Accuracy : {federated_acc:.4f}")
print("Coefficients :")
print(global_coef)
print("Intercept :")
print(global_intercept)


=========== 🔒 MODÈLE FÉDÉRÉ (AGRÉGATION SÉCURISÉE) ===========
Accuracy : 0.9825
Coefficients :
[-0.42586906 -0.42374269 -0.40610588 -0.4364777  -0.1878922   0.08177776
 -0.54109639 -0.67455419  0.0031552   0.11131585 -0.6445737   0.050447
 -0.43784198 -0.49676935  0.14535306  0.50870445  0.02914872  0.01947884
  0.24194873  0.52178696 -0.71839653 -0.93861213 -0.63094304 -0.65404472
 -0.5333236  -0.12383014 -0.6308644  -0.65072794 -0.82982855 -0.11336124]
Intercept :
[0.81544774]
